In [ ]:
import os
import csv
import numpy as np
import pandas as pd
import os
import re
import itertools
import collections
import tqdm
import random
import sys
import pickle

In [ ]:
# Ler Arquivos de Teste
arquivosTest = list(os.walk('Bases/TestePerplexidade/Train/'))[0][2]

corpus = ''
for a in arquivosTest:
        with open('Bases/TestePerplexidade/Train/' + a,'r', encoding='iso-8859-1') as f:
                corpus = corpus + f.read()

In [ ]:
def tokenize (txt):
    p = re.finditer ('(?:<TEXT>).*?(?:</TEXT>)',txt,flags=re.DOTALL|re.MULTILINE)
    for a in p:
        for x in re.finditer('[a-zA-ZâêîôûáéíóúàèìòùãẽĩõũçÁÉÍÓÚÀÈÌÒÙÃẼĨÕŨÂÊÎÔÛÇ]+|<TEXT>|</TEXT>|\.',a.group(),flags=re.DOTALL|re.MULTILINE):
            t = x.group()
            if t=='<TEXT>':
                yield '<s>'
                yield '<s>'
                yield '<s>'
            elif t == '</TEXT>':
                yield '</s>'
                yield '</s>'
                yield '</s>'
            elif t == '.':
                yield '</s>'
                yield '</s>'
                yield '</s>'
                yield '<s>'
                yield '<s>'
                yield '<s>'
            else:
                yield t.lower()

In [ ]:
# Total de Palavras
totalpalavras=0
for d in tokenize(corpus):
        totalpalavras+=1
print(totalpalavras)

In [ ]:
# Total de Palavras Distintas
inverseDict={}
total=0
for p in tokenize(corpus):
    if p not in inverseDict:
        inverseDict[p]=total
        total+=1
len(inverseDict)

In [ ]:
def slidingWindow(winsize):
    def itt(tokenizedText):
        iters=itertools.tee(tokenizedText,winsize)
        for x in range(winsize):
            for i in range(x):
                next(iters[x])
        while True:
            try:
                yield tuple([next(it) for it in iters])
            except StopIteration:
                return
    return itt

In [ ]:
f4 = slidingWindow(4)
f3 = slidingWindow(3)
f2 = slidingWindow(2)
quatro = f4(tokenize(corpus))
tres = f3(tokenize(corpus))
dois = f2(tokenize(corpus))
um = tokenize(corpus)

In [ ]:
total = sum(1 for _ in tokenize(corpus))
print (total)

In [ ]:
contador={}

In [ ]:
for a,b,c,d in tqdm.tqdm(zip(um,dois,tres,quatro),total=total):
    if a not in contador:
        contador[a]={None: 1}
    else:
        contador[a][None]+=1
        
    if b[1] not in contador[b[0]]:
        contador[b[0]][b[1]]={None: 1}
    else:
        contador[b[0]][b[1]][None]+=1
    
    if c[2] not in contador[c[0]][c[1]]:
        contador[c[0]][c[1]][c[2]] = {None: 1}
    else:
        contador[c[0]][c[1]][c[2]][None] += 1
        
    if d[3] not in contador[d[0]][d[1]][d[2]]:
        contador[d[0]][d[1]][d[2]][d[3]] = 1
    else:
        contador[d[0]][d[1]][d[2]][d[3]] += 1  

In [ ]:
with open('Bases/TestePerplexidade/recPerplexidade.pickle', 'wb') as handle:
    pickle.dump(contador, handle, protocol=pickle.HIGHEST_PROTOCOL)